In [95]:
import numpy as np

In [96]:
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

In [97]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f'{type(data)}은(는) 지원하지 않습니다.')

        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()             # 함수를 가져온다.
            x, y = f.input, f.output    # 함수의 입력과 출력을 가져온다.
            x.grad = f.backward(y.grad) # backward 메서드를 호출한다.

            if x.creator is not None:
                funcs.append(x.creator)

In [98]:
class Function:
    def __call__(self, input):
        x = input.data           # 데이터를 꺼낸다.
        y = self.forward(x)      # 구체적인 계산은 forward 메서드에서 한다.
        output = Variable(as_array(y))     # Variable 형태로 되돌린다.
        output.set_creator(self) # 출력 변수에 창조자를 설정한다.
        self.input = input       # 입력 변수를 기억(보관)한다.
        self.output = output     # 출력도 저장한다.
        return output

    def forward(self, x):
        raise NotImplementedError()

    def backward(self, gy):
        raise NotImplementedError()

In [99]:
class Square(Function):
    def forward(self, x):
        return x ** 2

    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx

class Exp(Function):
    def forward(self, x):
        return np.exp(x)

    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

In [100]:
def square(x):
    f = Square()
    return f(x)

def exp(x):
    f = Exp()
    return f(x)

In [101]:
x = Variable(np.array(0.5))
y = square(exp(square(x))) # 연속하여 적용
y.backward()
print(x.grad)

3.297442541400256


In [102]:
def numerical_diff(f, x, eps=1e-4):
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data) / (2 * eps)

In [107]:
import unittest

class SquareTest(unittest.TestCase):
    def test_forward(self):
        x = Variable(np.array(2.0))
        y = square(x)
        expected = np.array(4.0)
        self.assertEqual(y.data, expected)

    def test_backward(self):
        x = Variable(np.array(3.0))
        y = square(x)
        y. backward()
        expected = np.array(6.0)
        self.assertEqual(x.grad, expected)

    def test_gradient_check(self):
        x = Variable(np.random.rand(1)) # 무작위 입력값 생성
        y = square(x)
        y. backward()
        num_grad = numerical_diff(square, x)
        flg = np.allclose(x.grad, num_grad)
        self.assertTrue(flg)

unittest.main(argv=[''], verbosity=2, exit=False)

test_backward (__main__.SquareTest) ... ok
test_forward (__main__.SquareTest) ... ok
test_gradient_check (__main__.SquareTest) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.003s

OK
